<a href="https://colab.research.google.com/github/yanoo-o/AD_DOT/blob/main/AD_DOT_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Importing Packages**

We import the necessary packages to perform Multinomial Logistic Regression on the mode choice data.

The following packages are imported:
*   Pandas
*   Numpy
*   Scipy
*   sklearn
*   statsmodel.api
*   matplotlib.pyplot




In [2]:
#load all necessary libraries
import pandas as pd
import numpy as np
import scipy as scp
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import confusion_matrix

import statsmodels.api as sm
import matplotlib.pyplot as plt

# Binary Logistic Regression Model for Individual Level
This section creates a Logistic Regression model of mode choice on an individual level. The mode choices that was used in this analysis are:


*   0 - Non-Private
*   1 - Private

A total of 5923 individual observations (cleaned and organized) was used in this model. The following variables are used as predictors for the model:


1.   Control Variables

  *   Total number of families in the household
  *   Years of stay in the household

2.   Built Environment Variables (household TAZ)

  *   Average road speed
  *   Average road length
  *   Road density
  *   Land use Mix
  *   Population Density (Total Population/TAZ Area)
  *   Number of Transit Routes
  *   Number of Transit Stops

In [23]:
#read the dataset
#note that the csv file is space delimited
indiv = pd.read_csv('Individual_cleaned.csv',  delimiter=',')
print(indiv.head())

#Subsetting the data to remove classes with low frequency
# indiv1 = indiv[np.logical_and(indiv['modechoice']!=9,indiv['modechoice']!=6)]
# print(indiv1.head())

indiv_data = indiv.dropna()
# choice_class = indiv_data['modechoice'].value_counts()
# print(choice_class)
# print(indiv_data[indiv_data.columns[0]].count())


X = indiv_data.drop(['mode_main','id','income','hh_speed','hh_total_length','hh_total_pop','two_mode','main_mode','hh_buildup'], axis = 1)
print(X.head())
y = indiv_data['two_mode']

print(list(X.columns.values))

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.20, random_state = 5)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


model1 = LogisticRegression(random_state=0, solver='liblinear').fit(X_train, y_train)
preds = model1.predict(X_test)


params = model1.get_params()
print(params)

print('Intercept: \n', model1.intercept_)
print('Coefficients: \n', model1.coef_)

np.exp(model1.coef_)

logit_model=sm.Logit(y_train,sm.add_constant(X_train))
logit_model
result=logit_model.fit()
stats1=result.summary()
stats2=result.summary2()
print(stats1)
print(stats2)


                 id  travday   age  gender  income  hh_fourway  hh_speed  \
0   CAPI1213_182725        2   7.0     2.0     5.0    0.560000      49.0   
1   CAPI1213_182618        2   6.0     1.0     8.0    0.560000      49.0   
2  CAPI1213_1831019        2  10.0     1.0     9.0    0.560000      49.0   
3   CAPI1213_183918        2   9.0     1.0     8.0    0.560000      49.0   
4  CAPI1213_1861018        2  10.0     1.0     8.0    0.714286      43.0   

   hh_total_length  hh_ave_length  hh_density    hh_mix  hh_buildup  \
0         5693.791     247.556130    0.047830  0.365310         1.0   
1         5693.791     247.556130    0.047830  0.365310         1.0   
2         5693.791     247.556130    0.047830  0.365310         1.0   
3         5693.791     247.556130    0.047830  0.365310         1.0   
4         5432.201     208.930808    0.017105  0.544685         1.0   

   hh_total_pop  hh_pop_density  hh_routes  hh_stops  mode_main  main_mode  \
0        3235.0     27175.28267       

Create confusion matrix and calculate accuracy of the model

In [24]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_test,preds)
print('Confusion Matrix: \n', conf_matrix)

confmtrx = np.array(confusion_matrix(y_test, preds))
#Create DataFrame from confmtrx array
#rows for test: Male, Female, Infant designation as index
#columns for preds: male, predicted_female, predicted_infant as column

pd.DataFrame(confmtrx, index=['Non-Private','Private'],
columns=['Predicted Non-Private', 'Predicted Private'])

#Accuracy statistics

print('Accuracy Score:', metrics.accuracy_score(y_test, preds))

#Create classification report
class_report=classification_report(y_test, preds)
print(class_report)


Confusion Matrix: 
 [[  11  392]
 [   9 2549]]
Accuracy Score: 0.8645727794663964
              precision    recall  f1-score   support

         0.0       0.55      0.03      0.05       403
         1.0       0.87      1.00      0.93      2558

    accuracy                           0.86      2961
   macro avg       0.71      0.51      0.49      2961
weighted avg       0.82      0.86      0.81      2961



This section creates a Multinomial Logit Model for Household Data'

In [ ]:
house = pd.read_csv('household_cleaned.csv',  delimiter=',')
print(house.head())

#Subsetting the data to remove classes with low frequency
# house1 = house[np.logical_and(house['modechoice']!=9,house['modechoice']!=6)]
# print(house1.head())

house_data = house.dropna()
choice_class = house_data['two_mode'].value_counts()
print(choice_class)
print(house_data[house_data.columns[0]].count())

X = house_data.drop(['modechoice','fam_int','hh_location','hh_buildup','hh_total_length','hh_total_pop','main_mode','two_mode'], axis = 1)
print(X.head())
y = house_data['two_mode']

print(list(X.columns.values))

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.20, random_state = 5)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


model1 = LogisticRegression(random_state=0, penalty='none', solver='newton-cg').fit(X_train, y_train)
preds = model1.predict(X_test)

params = model1.get_params()
print(params)

print('Intercept: \n', model1.intercept_)
print('Coefficients: \n', model1.coef_)

np.exp(model1.coef_)

logit_model=sm.MNLogit(y_train,sm.add_constant(X_train))
logit_model
result=logit_model.fit()
stats1=result.summary()
stats2=result.summary2()
print(stats1)
print(stats2)

   tfam  fam_int  years  income  hh_location  hh_fourway  hh_speed  \
0   1.0      1.0    4.0     9.0          616    0.560000        49   
1   1.0      1.0    4.0     8.0          665    0.714286        43   
2   1.0      1.0    4.0     7.0          602    0.791667        34   
3   1.0      1.0    3.0     9.0          636    0.000000        62   
4   1.0      1.0    4.0     9.0          636    0.000000        62   

   hh_total_length  hh_ave_length  hh_density    hh_mix  hh_buildup  \
0         5693.791     247.556130    0.047830  0.365310           1   
1         5432.201     208.930808    0.017105  0.544685           1   
2        11461.813     133.276895    0.047477  0.568682           1   
3         6231.880     194.746250    0.269825  0.198505           1   
4         6231.880     194.746250    0.269825  0.198505           1   

   hh_total_pop  hh_pop_density  hh_routes  hh_stops  modechoice  main_mode  \
0          3235     27175.28267         31         4           1         

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'none', 'random_state': 0, 'solver': 'newton-cg', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
Intercept: 
 [2.02716042]
Coefficients: 
 [[-3.97843433e-01  2.51760425e-01  1.20669798e-03  3.36739537e-02
  -5.77865997e-03 -3.52469611e-04  3.95704864e+00 -6.60665141e-01
  -2.54096469e-05 -8.83859992e-03 -2.52505349e-02]]
Optimization terminated successfully.
         Current function value: 0.446251
         Iterations 6
                          MNLogit Regression Results                          
Dep. Variable:               two_mode   No. Observations:                 3928
Model:                        MNLogit   Df Residuals:                     3916
Method:                           MLE   Df Model:                           11
Date:                Wed, 21 Jun 2023   Pseudo R-squ.:                 0.044

/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(


Trip Level Multinomial Logistic Regression


In [ ]:
trip = pd.read_csv('trip_cleaned.csv',  delimiter=',')
print(trip.head())

#Subsetting the data to remove classes with low frequency
# trip11 = trip[np.logical_and(trip['mode_main']!=9,trip['mode_main']!=6)]
# trip1 = trip11[trip['mode_main']!=35]
# print(trip1.head())

trip_data = trip.dropna()
choice_class = trip_data['two_mode'].value_counts()
print(choice_class)
print(trip_data[trip_data.columns[0]].count())

X = trip_data.drop(['int_num','modechoice','t_start','t_end',
                    'hours','minutes','duration(m)','trip_start','trip_end','s_total_length','e_total_length',
                    's_buildup','e_buildup','s_total_pop','e_total_pop','main_mode','two_mode','s_speed','e_speed',
                    's_density','e_density'], axis = 1)
print(X.head())
y = trip_data['two_mode']

print(list(X.columns.values))

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.20, random_state = 5)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


model1 = LogisticRegression(random_state=0, penalty='none', solver='newton-cg').fit(X_train, y_train)
preds = model1.predict(X_test)

params = model1.get_params()
print(params)

print('Intercept: \n', model1.intercept_)
print('Coefficients: \n', model1.coef_)

np.exp(model1.coef_)

logit_model=sm.MNLogit(y_train,sm.add_constant(X_train))
logit_model
result=logit_model.fit()
stats1=result.summary()
stats2=result.summary2()
print(stats1)
print(stats2)



       int_num  travday  age  gender  income      t_start        t_end  hours  \
0  CAPI234_298      4.0  2.0     1.0     1.0  11:30:00 AM  11:45:00 AM    0.0   
1  CAPI234_298      4.0  2.0     1.0     1.0   5:30:00 PM   6:00:00 PM    0.0   
2  CAPI234_299      4.0  8.0     1.0    14.0   8:00:00 AM   8:25:00 AM    0.0   
3  CAPI234_299      4.0  8.0     1.0    14.0  12:30:00 PM  12:50:00 PM    0.0   
4  CAPI234_299      4.0  7.0     2.0     1.0   6:00:00 PM   6:05:00 PM    0.0   

   minutes  duration(m)  ...  s_pop_density  e_total_pop  e_pop_density  \
0     15.0         15.0  ...    3831.670551         31.0      72.716866   
1     30.0         30.0  ...      72.716866       2670.0    3831.670551   
2     25.0         25.0  ...    3831.670551        212.0     104.793305   
3     20.0         20.0  ...     104.793305       2670.0    3831.670551   
4      5.0          5.0  ...    3831.670551       2670.0    3831.670551   

   s_routes  s_stops  e_routes  e_stops  modechoice  main_mode

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(


{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'none', 'random_state': 0, 'solver': 'newton-cg', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
Intercept: 
 [2.30498513]
Coefficients: 
 [[ 7.68112135e-02 -1.80144597e-01  5.67926372e-01  7.67058726e-02
  -1.62166778e-01 -1.90252234e-04 -1.24169832e-01 -2.26194006e-04
  -4.26845759e-02 -1.02536021e-01 -1.56459585e-05 -1.63062025e-05
  -3.13208196e-03 -9.25404970e-03 -2.92511661e-03 -1.02981697e-02]]
Optimization terminated successfully.
         Current function value: 0.369924
         Iterations 6
                          MNLogit Regression Results                          
Dep. Variable:               two_mode   No. Observations:                28409
Model:                        MNLogit   Df Residuals:                    28392
Method:                           MLE   Df Model:                        